# **This notebooks aims to execute the full training process of our model from the data preprocessing to the training phase**

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#We need to downgrade this keras version to be able to use some functions
pip uninstall keras

Uninstalling Keras-2.2.5:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/Keras-2.2.5.dist-info/*
    /usr/local/lib/python3.6/dist-packages/docs/*
    /usr/local/lib/python3.6/dist-packages/keras/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.6/dist-packages/docs/md_autogen.py
    /usr/local/lib/python3.6/dist-packages/docs/update_docs.py
Proceed (y/n)? y
  Successfully uninstalled Keras-2.2.5


In [0]:
pip install keras==2.1.5

     |████████████████████████████████| 337kB 9.2MB/s 


In [0]:
import keras
print(keras.__version__)
#We check the keras version to make sure that the downgrading operation went well.

Using TensorFlow backend.


2.1.5


In [0]:
"""
Train the YOLO model for the Detection and classification of “salad & fruit bar” tasks.

"""

#Librairies importation
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/Technical test - Trayvisor')
import numpy as np
from datetime import datetime
from random import shuffle
import keras.backend as K
from keras.layers import Input, Lambda
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from yolo3.model import preprocess_true_boxes, yolo_body, yolo_loss
from yolo3.utils import get_random_data


def BB_extraction(data,image_path):
    '''Extracts image name,label and Bounding Box coordinates

    Parameters
    ----------
    -data: .npy file containing the data that'll be used to train the CNN.
    -image_path: str, is the path where we find the images

    Returns
    -------
    -Data_info: list, shape like [[image1 absolute path,box11,..box1m],...[image n absolute path, boxn1,..boxn] ]
     where box=[xmin,ymin,xmax,ymax,class_index], contains the main information about our dataset.
    -classes: list, contains the name of the various classes found in our dataset'''

    classes=[]
    Data_info=[]
    bv=0
    sv=0
    for number in range(len(data)):
        liste=[]
        liste.append(image_path+'/'+data[number]['name'])
                      
        for i in range(len(data[number]['boxes'])):
            cord=data[number]['boxes'][i]['box']   
            cord[2]+=cord[0]
            cord[3]+=cord[1]
            class_name=data[number]['boxes'][i]['id']
                      
            if(class_name not in classes):
                classes.append(class_name)
                
            if(class_name=='big_vrac'):
                bv+=1
            else:
                sv+=1
                
            cord.append(classes.index(class_name))
            
            liste.append(cord)
            
        Data_info.append(liste)
    print('Big vrac and small vrac occurences respectively:',bv,sv)
    
    del data
            
    return Data_info,classes

def test_train_split(data,data_path,test_ratio=0.2):

    '''Splits the dataset into train set and test set and save the train and test sets in a directory.

    Parameters
    ----------
    -data: list, shape like [[image1 absolute path,box11,..box1m],...[image n absolute path, boxn1,..boxn] ]
     where box=[xmin,ymin,xmax,ymax,class_index], contains the main information about our dataset.
    -data_path: str, is the path where we want to save our train and test sets.
    test_ratio: float, between 0 and 1, is the percentage (of the whole dataset) that will be used to create the test set.

    Returns
    -------
    -train_set: list, corresponds to the training set.
    -test_set: list, corresponds to the test set.'''

    train_set=[]
    test_set=[]
    shuffle(data)
    num=int(len(data)-len(data)*test_ratio)
    train_set=data[:num]
    test_set=data[num:]
        
    file_path=new_file_name(data_path)
    os.mkdir(file_path)
    np.save(file_path+'/test.npy',np.array(test_set))
    np.save(file_path+'/train.npy',np.array(test_set))
        
    return train_set, test_set

def new_file_name(path):
  '''Creates a folder name based on the current date and hour.

    Parameters
    ----------
    -path: str, is the path where a new folder will be created
    Returns
    -------
    -file_path: str, corresponds to the name of a new folder'''
    
  date=datetime.now()
  file_path=path+'/dataset'+str(date.year)+str(date.month)+str(date.day)+str(date.hour)+str(date.minute)+str(date.second)
  return file_path

def get_anchors(anchors_liste):
    '''Creates anchors that will be used for the detection task'''
    return np.array(anchors_liste, dtype='float').reshape(-1, 2)   

def create_model(input_shape, anchors, num_classes, load_pretrained=True, freeze_body=2, weights_path='./yolo_weights.h5'):
    '''Creates a CNN model (YOLO model).

    Parameters
    ----------
    -input_shape: tuple, represents the shape of the image that will be used as the input of our model.
    -anchors: array, contains the anchors that will be used for the detection task.
    -num_classes: int, is the number of classes without the background.
    -load_pretrained: Bool, says if we want to use the weights of a previous trained model as a starting point or not.
    -freeze_body: int, specifes the number of layers we want to freeze during the training phase.
    -weights_path: str, is the path where we find the pretrained weights.
    Returns
    -------
    -A CNN model'''
  
    K.clear_session() # get a new session
    image_input = Input(shape=(None, None, 3))
    h, w = input_shape
    num_anchors = len(anchors)

    y_true = [Input(shape=(h//{0:32, 1:16, 2:8}[l], w//{0:32, 1:16, 2:8}[l], \
        num_anchors//3, num_classes+5)) for l in range(3)]

    model_body = yolo_body(image_input, num_anchors//3, num_classes)
    print('Create YOLOv3 model with {} anchors and {} classes.'.format(num_anchors, num_classes))

    if load_pretrained:
        model_body.load_weights(weights_path, by_name=True, skip_mismatch=True)
        print('Load weights {}.'.format(weights_path))
        if freeze_body in [1, 2]:
            # Freeze darknet53 body or freeze all but 3 output layers.
            num = (185, len(model_body.layers)-3)[freeze_body-1]
            for i in range(num): model_body.layers[i].trainable = False
            print('Freeze the first {} layers of total {} layers.'.format(num, len(model_body.layers)))

    model_loss = Lambda(yolo_loss, output_shape=(1,), name='yolo_loss',
        arguments={'anchors': anchors, 'num_classes': num_classes, 'ignore_thresh': 0.5})(
        [*model_body.output, *y_true])
    model = Model([model_body.input, *y_true], model_loss)

    return model

def data_generator(annotation_lines, batch_size, input_shape, anchors, num_classes):
    '''Generates the data that will be fed into the CNN'''
    n = len(annotation_lines)
    i = 0
    while True:
        image_data = []
        box_data = []
        for b in range(batch_size):
            if i==0:
                np.random.shuffle(annotation_lines)
            image, box = get_random_data(annotation_lines[i], input_shape, random=True)
            image_data.append(image)
            box_data.append(box)
            i = (i+1) % n
        image_data = np.array(image_data)
        box_data = np.array(box_data)
        y_true = preprocess_true_boxes(box_data, input_shape, anchors, num_classes)
        yield [image_data, *y_true], np.zeros(batch_size)

def data_generator_wrapper(annotation_lines, batch_size, input_shape, anchors, num_classes):
  '''Generates the data that will be fed into the CNN'''
  n = len(annotation_lines)
  if n==0 or batch_size<=0: return None
  return data_generator(annotation_lines, batch_size, input_shape, anchors, num_classes)

In [0]:
if __name__ == '__main__':

  #We define a directory 
    log_dir = '/content/drive/My Drive/Colab Notebooks/Technical test - Trayvisor/model_weights'
    
    #We define the Anchor size
    ANCHOR_SIZE=[10,13,16,30,33,23,30,61,62,45,59,119,116,90,156,198,373,326]
    
    #data_path is the path where we find the trayvisor_test_db.npy which contains the data
    data_path='/content/drive/My Drive/Colab Notebooks/Technical test - Trayvisor/trayvisor_test_db.npy'
    
    #path is the path where we find the images
    path='/content/drive/My Drive/Colab Notebooks/Technical test - Trayvisor/images'

    #we define a path where we'll save the test set and train set
    save_dataset_path='/content/drive/My Drive/Colab Notebooks/Technical test - Trayvisor'
    
    #we get the data from the .npy file 
    label=np.load(data_path,allow_pickle=True)
    
    #We extract the BB coordinates and the classes
    data_info,class_names=BB_extraction(label,path)
     
    num_classes = len(class_names)
    
    anchors = get_anchors(ANCHOR_SIZE)
    #We split our dataset into test and train sets
    train,test=test_train_split(data_info,save_dataset_path)

    input_shape = (416,416) # multiple of 32, hw
    
    #We then create the model as well as the checkpoints 
    model = create_model(input_shape, anchors, num_classes, freeze_body=2, weights_path='/content/drive/My Drive/Colab Notebooks/Technical test - Trayvisor/yolo_weights.h5')
    logging = TensorBoard(log_dir=log_dir)
    checkpoint = ModelCheckpoint(log_dir + '/ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5',
        monitor='val_loss', save_weights_only=True, save_best_only=True, period=3)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)
    early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1)

    # Train with frozen layers first, to get a stable loss.
    if True:
        model.compile(optimizer=Adam(lr=1e-3), loss={
            # use custom yolo_loss Lambda layer.
            'yolo_loss': lambda y_true, y_pred: y_pred})

        batch_size = 32
        print('Train on {} samples, val on {} samples, with batch size {}.'.format(len(train), len(test), batch_size))
        model.fit_generator(data_generator_wrapper(train, batch_size, input_shape, anchors, num_classes),
                steps_per_epoch=max(1, len(train)//batch_size),
                validation_data=data_generator_wrapper(test, batch_size, input_shape, anchors, num_classes),
                validation_steps=max(1, len(test)//batch_size),
                epochs=50,
                initial_epoch=0,
                callbacks=[logging, checkpoint])
        model.save_weights(log_dir + '/trained_weights_stage_1.h5')

    # Unfreeze and continue training, to fine-tune.
    if True:
        for i in range(len(model.layers)):
            model.layers[i].trainable = True
        model.compile(optimizer=Adam(lr=1e-4), loss={'yolo_loss': lambda y_true, y_pred: y_pred}) # recompile to apply the change
        print('Unfreeze all of the layers.')

        batch_size = 8 
        print('Train on {} samples, val on {} samples, with batch size {}.'.format(len(train), len(test), batch_size))
        model.fit_generator(data_generator_wrapper(train, batch_size, input_shape, anchors, num_classes),
            steps_per_epoch=max(1, len(train)//batch_size),
            validation_data=data_generator_wrapper(test, batch_size, input_shape, anchors, num_classes),
            validation_steps=max(1, len(test)//batch_size),
            epochs=100,
            initial_epoch=50,
            callbacks=[logging, checkpoint, reduce_lr, early_stopping])
        model.save_weights(log_dir + '/trained_weights_final.h5') 